# Fase III - Campanha 02: 802.11ac + QoS

## Objetivos: 

O objetivo da etapa atual é estudar o comportamento do protocolo **802.11ac** em conjunto com o **Enhanced Distribution Channel Access (EDCA)** e múltiplos serviços, representados pelas **Access Categories (AC)**, por meio de simulações realizadas com o auxílio do ns-3.

## Implementação:

O script de simulação foi construído utilizando o **ns-3.30.1** e tem como base o exemplo [wifi-multi-tos.cc](https://www.nsnam.org/docs/release/3.30/doxygen/wifi-multi-tos_8cc.html) proveniente do mófulo Wi-Fi, sendo encontrado em **_~/ns-allinone-3.30.1/ns-3.30.1/examples/wireless_**. O código padrão possui a seguinte descrição:

![descrição padrão wifi-multi-tos.cc](./FIGS/default_description.png)

Como o exemplo em questão configura uma rede semelhante ao cenário de simulação desejado. Algumas alterações foram realizadas para que o mesmo atendesse as nossas necessidades. Em seguida serão apresentadas as principais alterações efetuadas no código principal.

### Alteração do padrão PHY Wi-Fi:

No script padrão a simulação é realizada utilizando o padrão PHY 802.11n na frequência de 5GHz.

![padrão PHY original](./FIGS/default_standard.png)

O padrão Wi-Fi é facilmente modificado por meio da função **_SetStandard_**, proveniente do **WifiHelper**. Substituimos o valor anterior para **WIFI_PHY_STANDARD_80211ac** que representa o novo padrão desejado.

![padrão PHY modificado](./FIGS/modified_standard.png)


### Alteração do Algoritmo de adaptação de taxa:

Não utilizamos algoritmos de adaptação de taxa no cenário, com isso o **ConstantRateWifiManager** é mantido. A única modificação realizada foi o uso dos **Very High Troughput MCS (VHT MCS)** apropriados para o padrão 802.11ac.

![wifistationmanager modificado](./FIGS/Modified_Manager.png)

### Configurando a MAC:

Para configurar a nossa camada MAC foi é utilizado o **WifiMacHelper** que fica responsável por modificar e installar parâmetros como o tipo da MAC (AP, STA ou Ad-Hoc) e o suporte a qualidade de serviço (QoS) nos nós desejados. Como estamos trabalhando com um cenário que necessita de QoS, habilitamos o seu suporte através do atributo **"QosSupported"**, onde por padrão é deshabilitado.

![MAC modificado](./FIGS/modified_MAC.png)


### Configurando a Mobilidade:

A nova configuração de mobilidade é simples, o AP está localizado na posição (0,0,0) enquanto a STA (ou STAs) são alocados em posições aleatórias e nenhum nó se move durante a simulação.

![Mobilidade modificado](./FIGS/modified_mobility2.png)

### Configurando o endereçamento dos nós:
O **InternetStackHelper** foi utilizado para agregar as funcionalidades IP/TCP/UDP aos nós existentes no cenário. Setamos o endereço IP do AP como 192.168.1.1 e os subsequentes são alocados as STAs.

![Internet_stack](./FIGS/modified_internetstack.png)

### Configurando as Aplicações:
Finalizando a implementação do cenário configuramos as aplicações utilizadas na simulação com o auxílio do **OnOffHelper** e **PacketSinkHelper** com os conhecimentos obtidos em fases passadas do treinamento. O desafio encontrado nessa etapa foi aprender a manipular as quatro categorias de acesso (ACs) utilizadas no EDCA (Best Effort, Background, Video e Voice).

De acordo com a [documentação do módulo Wi-Fi](https://www.nsnam.org/docs/release/3.30/models/html/wifi-user.html#selection-of-the-access-category-ac), a partir do ns-3.26 a seleção da AC não é realizada por meio de uma **QosTag** adicionada ao pacote enviado, atualmente essa seleção é feita alterando o campo **Type of Service (ToS)**  no header IP do pacote. Na prática, essa modificação é realizada criando um novo socket de destino ( _SinkSocket_ ) onde especificamos o ToS desejado. Os valores de ToS que representam cada AC foram encontrados no seguinte [exemplo](https://www.nsnam.org/docs/release/3.30/doxygen/wifi-ac-mapping-test-suite_8cc.html) e foram setados por meio da função _SetTos_. Utilizamos uma aplicação OnOff para transmitir as quatro streams de tráfego, oferecendo 20 Mbps para cada uma.

![ToS_Table](./FIGS/ToStable.png)

* **Best Effort (AC_BE):** 0x70
* **Background (AC_BK):** 0x28
* **Video (AC_VI):** 0xb8
* **Voice (AC_VO):** 0xc0

![downlink application](./FIGS/application_modifieddl.png)

### Resultados Iniciais: 

Para uma simulação variando a carga de STAs de 1 a 30, oferecendo 20 Mbps para cada stream de tráfego e um tempo de simulação de 100 segundos, obtemos o seguinte resultado.

![first_result](./FIGS/wifi_edca_definitive_downlink_Tput.png)

Como podemos observar, a prioridade esperada (AC_VO > AC_VI > AC_BE > AC_BK) não foi totalmente respeitada, havendo uma inversão entre as categorias de acesso de voz e vídeo. Em conjunto a inversão de prioridades, é possível observar que com a taxa oferecida para as ACs já ocorre uma saturação no tráfego da categoria de acesso background. Esse resultado inicial foi o pontapé para iniciar uma investigação mais profunda sobre o EDCA no ns-3.

## Novas investigações:

Uma nova rodada de investigações foi elaborada para tentar entender o comportamento do EDCA no ns-3, os testes realizados são listados a seguir:

* Simulação utilizando os parâmetros do **802.11e** (padrão do ns-3)
* Simulação utilidando os parâmetros do **802.11p**
* Influência dos parâmetros de cada categoria de acesso (AIFSN, CWmin, CWmax e TXOP):
    * Influência do **AIFSN**
        * **AIFSN** seguindo o padrão 802.11e
        * **CWmin** igual a 15 para todas as ACs
        * **CWmax** igual a 1023 para todas as ACs
        * **TXOP** igual a 0 para todas as ACs
    * Influência da **Contention Window (CW)**
        * **AIFSN** igual a 2 para todas as ACs
        * **CWmin** seguindo o padrão 802.11e
        * **CWmin** seguindo o padrão 802.11e
        * **TXOP** igual a 0 para todas as ACs
    * Influência da **Transmission Opportunity (TXOP)**
        * **AIFSN** igual 2 para todas as ACs
        * **CWmin** igual a 15 para todas as ACs
        * **CWmax** igual a 1023 para todas as ACs
        * **TXOP** seguindo o padrão 802.11e

![EDCAparameters](./FIGS/EDCAparameters_table.png)

Ao ativar o suporte a QoS na camada MAC o ns-3 cria quatro filas de pacotes diferentes (**BE_Txop**, **BK_Txop**, **VI_Txop**, **VO_Txop**), uma para cada categoria de acesso. Seguindo os Config Paths sugeridos pela [documentação](https://www.nsnam.org/docs/release/3.30/doxygen/classns3_1_1_regular_wifi_mac.html) do ns-3, é possível modificar os atributos necessários nos novos cenários.

![ConfigPaths](./FIGS/QueuesConfig_Path.png)
![ExistingQueues](./FIGS/txop_queues.png)
![QueueConfigPaths](./FIGS/Queues_Configs.png)

### Definição da nova taxa oferecida

Para definir a nova taxa oferecida para cada categoria de acesso foi realizada uma breve rodada de simulações, a taxa oferecida foi variada partindo de 1 Mbps até 100 Mbps com apenas uma STA para descobrir em que ponto a taxa de cada categoria começa a saturar, assim encontrando um ponto comum para todas as ACs. Essa etapa foi realizada para cada cenário estabelecido anteriormente. Os resultados obtidos são exibidos a seguir.

![saturation802.11e](./FIGS/wifi_edca_saturation_80211e_Tput.png)
![saturation802.11p](./FIGS/wifi_edca_saturation_80211p_Tput.png)
![saturationaifsn](./FIGS/wifi_edca_saturation_aifsn_Tput.png)
![saturationcw](./FIGS/wifi_edca_saturation_cw_Tput.png)
![saturationtxop](./FIGS/wifi_edca_saturation_txop_Tput.png)

Observando os gráficos anteriores definimos uma taxa em comum para todas as ACs em 10 Mbps.
